- https://docs.julialang.org/en/v1/manual/performance-tips/#Access-arrays-in-memory-order,-along-columns-1
- performance critical code should be inside a function.
- and avoid untyped global variables.

In [1]:
# global names are frequently constants.

const DEFAULT_VAL = 0

# If a global is known to always be of the same type; the type should be annotated.
global x = rand(1000)

function loop_over_global()
    s = 0.0
    for i in x::Vector{Float64}
        s+= i
    end
    return s
end

loop_over_global (generic function with 1 method)

1. 전역 변수를 직접 사용하기보단, argument를 함수에 전달할 것.
loop_over_global() 함수는 전역함수 x만을 처리할 수 있지만, sum_vector(my_vec)처럼 벡터를 인자로 받으면 어떤 벡터는 이 함수에 전달가능.

2. REPL의 모든 코드는 전역 공간에서 실행된다.
X = 1.0과 global x = 1.0은 동일한 효과.

In [2]:
# Measure performance with @time.

x = rand(1000)

function sum_global()
    s = 0.0
    for i in x
        s += 1
    end
    return s
end;

@time sum_global()

  0.018069 seconds (2.77 k allocations: 66.719 KiB, 99.18% compilation time)


1000.0

In [3]:

@time sum_global()

  0.000100 seconds (2.49 k allocations: 54.531 KiB)


1000.0

1> 첫 번째 실행에선 compile 과정이 포함되어 시간이 오래 걸림.
2> 두 번째 실행에선 compile 과정이 포함되지 않고, 실제 함수의 실행 성능만을 보임.

즉, 예상치 못한 메모리 할당은 거의 항상 코드에 성능 문제가 있다는 것을 보여줌.

stack이 아닌 heap(크기가 변할 수 있는 배열이나 타입 불안정 공간)에 할당하는 경우 효율적으로 처리를 못함.

따라서 전역 변수가 아닌 인자를 사용하는게 현명하다.


In [4]:
function sum_arg(x)
    s=0.0
    for i in x
        s+=1
    end
    return s
end

@time sum_arg(x)

  0.014685 seconds (3.69 k allocations: 178.545 KiB, 99.54% compilation time)


1000.0

In [6]:
@time sum_arg(x)

  0.000012 seconds (1 allocation: 16 bytes)


1000.0

In [7]:
a = Real[]
Real[]

Real[]

In [8]:
push!(a, 1); push!(a, 2.0); push!(a, π) 

3-element Vector{Real}:
 1
 2.0
 π = 3.1415926535897...

In [11]:
c = Float64[]
push!(c, 1); push!(c, 2.0); push!(c, π)

3-element Vector{Float64}:
 1.0
 2.0
 3.141592653589793

In [12]:
# Type declarations
struct MyAmbigiousType
    a
end